In [ ]:
## Question 1 ##

In [1]:
import os, spacy

In [2]:
nlp = spacy.load("fr_core_news_sm")

In [8]:
def analyse_fichier(path):
    text = open(path, encoding="utf-8").read()
    doc = nlp(text)
    nb_mots = len([t for t in doc if not t.is_punct])
    nb_phrases = len(list(doc.sents))
    return nb_mots, nb_phrases

if __name__ == "__main__":
    dossier = "CorpusARIA/TXT"
    total_mots, total_phrases = 0, 0
    fichiers = os.listdir(dossier)
    n_fichiers = len(fichiers)

    for f in fichiers:
        path = os.path.join(dossier, f)
        mots, phrases = analyse_fichier(path)
        total_mots += mots
        total_phrases += phrases
        print(f"{f}: {mots} mots, {phrases} phrases")

    moy_mots = round(total_mots / n_fichiers)
    moy_phrases = round(total_phrases / n_fichiers)

    print(f"\nTotal: {total_mots} mots, {total_phrases} phrases")
    print(f"Moyenne par fichier: {moy_mots} mots, {moy_phrases} phrases")


event-200.txt: 89 mots, 6 phrases
event-117.txt: 108 mots, 6 phrases
event-76.txt: 66 mots, 4 phrases
event-207.txt: 40 mots, 3 phrases
event-110.txt: 242 mots, 11 phrases
event-71.txt: 418 mots, 19 phrases
event-275.txt: 54 mots, 3 phrases
event-162.txt: 120 mots, 10 phrases
event-209.txt: 101 mots, 6 phrases
event-119.txt: 241 mots, 10 phrases
event-78.txt: 151 mots, 9 phrases
event-272.txt: 74 mots, 3 phrases
event-165.txt: 89 mots, 5 phrases
event-31.txt: 280 mots, 16 phrases
event-150.txt: 166 mots, 6 phrases
event-181.txt: 70 mots, 7 phrases
event-247.txt: 75 mots, 7 phrases
event-36.txt: 109 mots, 5 phrases
event-157.txt: 112 mots, 10 phrases
event-186.txt: 61 mots, 4 phrases
event-240.txt: 118 mots, 4 phrases
event-38.txt: 157 mots, 8 phrases
event-159.txt: 170 mots, 10 phrases
event-188.txt: 54 mots, 4 phrases
event-44.txt: 494 mots, 28 phrases
event-125.txt: 159 mots, 12 phrases
event-95.txt: 421 mots, 22 phrases
event-232.txt: 107 mots, 5 phrases
event-43.txt: 137 mots, 7 ph

event-170.txt: 86 mots, 5 phrases
event-267.txt: 89 mots, 6 phrases
event-16.txt: 278 mots, 9 phrases
event-177.txt: 131 mots, 7 phrases
event-260.txt: 80 mots, 5 phrases
event-64.txt: 374 mots, 24 phrases
event-105.txt: 103 mots, 8 phrases
event-212.txt: 59 mots, 4 phrases
event-18.txt: 227 mots, 12 phrases
event-179.txt: 113 mots, 7 phrases
event-269.txt: 79 mots, 3 phrases
event-63.txt: 364 mots, 14 phrases
event-102.txt: 149 mots, 10 phrases
event-215.txt: 128 mots, 7 phrases
event-278.txt: 98 mots, 8 phrases
event-204.txt: 73 mots, 4 phrases
event-72.txt: 227 mots, 16 phrases
event-113.txt: 127 mots, 5 phrases
event-203.txt: 98 mots, 6 phrases
event-75.txt: 41 mots, 1 phrases
event-114.txt: 115 mots, 7 phrases
event-168.txt: 94 mots, 5 phrases
event-271.txt: 59 mots, 4 phrases
event-166.txt: 84 mots, 5 phrases
event-276.txt: 66 mots, 4 phrases
event-161.txt: 137 mots, 8 phrases
event-154.txt: 145 mots, 7 phrases
event-35.txt: 529 mots, 32 phrases
event-243.txt: 179 mots, 10 phrase

In [ ]:
# Total: 46767 mots, 2548 phrases
# Moyenne par fichier: 164 mots, 9 phrases

In [ ]:
## Question 2 ##

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [4]:
# chargement des labels (conséquences à gros grain)

def lire_consequence_gros_grain(path_meta):
    with open(path_meta, encoding="utf-8") as f:
        ligne = f.readline().strip()
        champs = ligne.split(";")
        if len(champs) >= 16:
            return champs[15].strip()
        return None

texts, labels = [], []
meta_dir = "CorpusARIA/METADATA3"
txt_dir = "CorpusARIA/TXT"

for meta in os.listdir(meta_dir):
    path_meta = os.path.join(meta_dir, meta)
    label = lire_consequence_gros_grain(path_meta)
    txt_path = os.path.join(txt_dir, meta.replace("_md.txt", ".txt"))
    if os.path.exists(txt_path) and label:
        text = open(txt_path, encoding="utf-8").read()
        texts.append(text)
        labels.append(label)

print(f"{len(texts)} fichiers chargés.")
pd.Series(labels).value_counts()


285 fichiers chargés.


CONSÉQUENCES ENVIRONNEMENTALES                                                   110
CONSÉQUENCES ÉCONOMIQUES                                                          75
Inconnue                                                                          42
CONSÉQUENCES ÉCONOMIQUES,CONSÉQUENCES SOCIALES                                    39
CONSÉQUENCES ÉCONOMIQUES,CONSÉQUENCES ENVIRONNEMENTALES                           13
CONSÉQUENCES ÉCONOMIQUES,CONSÉQUENCES SOCIALES,CONSÉQUENCES ENVIRONNEMENTALES      6
dtype: int64

In [21]:
# vectorisation

from sklearn.feature_extraction.text import CountVectorizer


cvect = CountVectorizer(ngram_range=(1,2), max_features=5000)
#vect = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X = cvect.fit_transform(texts)
print("Dimensions du vecteur :", X.shape)


Dimensions du vecteur : (285, 5000)


In [22]:
# validation croisée 

from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, labels, cv=cv, scoring='f1_macro')
print("F1-macro (5-fold) : {:.2f} ± {:.2f}".format(scores.mean(), scores.std()))


F1-macro (5-fold) : 0.46 ± 0.02


In [23]:
# entrainement et eval du modele

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


                                                         precision    recall  f1-score   support

                         CONSÉQUENCES ENVIRONNEMENTALES       0.96      1.00      0.98        27
                               CONSÉQUENCES ÉCONOMIQUES       0.59      0.71      0.65        14
CONSÉQUENCES ÉCONOMIQUES,CONSÉQUENCES ENVIRONNEMENTALES       0.00      0.00      0.00         1
         CONSÉQUENCES ÉCONOMIQUES,CONSÉQUENCES SOCIALES       0.62      0.56      0.59         9
                                               Inconnue       0.50      0.33      0.40         6

                                               accuracy                           0.77        57
                                              macro avg       0.54      0.52      0.52        57
                                           weighted avg       0.75      0.77      0.76        57



/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
"""1ere Partie : 

Le corpus contient 47 767 mots et 2548 phrases.
Chaque fichier contient en moyenne 164 mots et 9 phrases


2eme Partie : 

Les classes fréquentes (environnementales, économiques) sont bien reconnues.
Les classes avec deux conséquences sont peu représentées, d’où une précision et un rappel faibles.
La classe "Inconnue" est difficile à distinguer, avec des performances modestes.
Le modèle atteint 77 % de précision, performant sur les catégories bien repprésentées mais limité par le déséquilibre du corpus.

La classe (CONSÉQUENCES ÉCONOMIQUES,CONSÉQUENCES ENVIRONNEMENTALES) est sous représentée (support = 1) donc elle n'est jamais prédite.
"""